# Uncertainty Quantification

In fields such as engineering, physics, and applied mathematics, simulation models serve as crucial tools for predicting real-world phenomena. These models typically rely on precise parameterization, including aspects like constitutive models and boundary conditions. However, in practical applications, these parameters are often unknown due to insufficient experimental data. To ensure accurate predictions despite this uncertainty, it is essential to incorporate these unknowns into the modeling process. This is precisely the focus of the field of uncertainty quantification, which aims to systematically address and manage the uncertainties inherent in simulation models.

In this tutorial, we'll have a look at forward uncertainty quantification. Here the goal is to model the uncertainty in quantity of interest $y$ by propagating uncertainties in input $\theta$ through the model $f(\theta)$:
$$u = f(\theta)$$

Here, we assume the input $\theta$ is uncertain, which we describe with a random variable using a probability distribution $p(\theta)$. As a consequence, the outputs of the model $m$ are random variables as well, following a distribution $p(u)$. *Random in, random out.*

## An example

Let's look at an example, where the function $f$ is the solution of a partial differential equation.

### The model

On a domain $\Omega=[0,1] \times [0,1]$, the Poisson equation is given by:

$$-\Delta u = s$$

where $u$ is the solution field and $s$ the heterogenous source term. To make the problem well-posed, we'll apply the boundary conditions:
$$u = 0 \text{ for } x \in \partial \Omega$$ 

To solve the partial differential equation, we employ a finite element approach with linear elements using [scikit-fem](https://github.com/kinnala/scikit-fem/tree/master) (this tutorial is inspired by their [tutorial](https://scikit-fem.readthedocs.io/en/latest/listofexamples.html#poisson-equation))


## The uncertainties
For this system, we assume the source term $s$ is modelled by
$$ s(x,y,x_s,y_s) = \exp\left( -\frac{1}{2}\frac{(x-x_s)^2 + (y-y_s)^2}{0.1^2}\right)$$
where the coordinates $x_s$ and $y_s$ are uncertain. The source center is defined via the joint distribution
$$p(x_s,y_s) = \mathcal{B}(x_s|2,5) \mathcal{B}(y_s|4,3) $$
where $\mathcal{B}(\circ|a,b)$ is a beta distribution with shape parameters $a$ and $b$. Here, we assume the parameters are independent of each other, i.e., $p(x_s,y_s) = p(x_s)p(y_s)$, where $p(x_s)$ and $p(y_s)$ are called marginal distributions.


In [ ]:
from skfem import Basis, BilinearForm, ElementTriP1, LinearForm, MeshTri, enforce, solve
from skfem.helpers import dot, grad

mesh = MeshTri().refined(6)


def poisson_pde(source_x, source_y, source_term):

    # Set discretization
    e = ElementTriP1()
    basis = Basis(mesh, e)

    @BilinearForm
    def laplace(u, v, _):
        return dot(grad(u), grad(v))

    @LinearForm
    def rhs(v, w):
        # Source term
        return source_term(w.x[0], w.x[1], source_x, source_y) * v

    # Stiffness matrix
    A = laplace.assemble(basis)

    # Right-hand side
    b = rhs.assemble(basis)

    # Enforce Dirichlet boundary conditions
    A, b = enforce(A, b, D=mesh.boundary_nodes())

    # Solve
    solution = solve(A, b)

    return solution


def plot_to_axis(field, ax):
    mesh.plot(field, ax=ax)
    ax.axis("equal")
    ax.set_aspect("equal", "box")
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")


In [ ]:
# Let's define the distributions

from queens.distributions import Beta

x_s = Beta(0, 1, 2, 5)
y_s = Beta(0, 1, 4, 3)

In [ ]:
# Let's plot the probability density functions (PDF)

import numpy as np
import matplotlib.pyplot as plt

# x, y in [0,1]

coordinate = np.linspace(0, 1, 100)

plt.plot(coordinate, x_s.pdf(coordinate), "r-", label="$p(x_s)$")
plt.plot(coordinate, y_s.pdf(coordinate), "b-", label="$p(y_s)$")

plt.xlabel("$x_s$ or $y_s$")
plt.ylabel("Probability density functions")
plt.title("Marginal distributions")
plt.legend()
plt.show()

In [ ]:
# Let's define the parameters using QUEENS and plot the joint distribution

from queens.parameters import Parameters

parameters = Parameters(x_s=x_s, y_s=y_s)

xx, yy = np.meshgrid(coordinate, coordinate)

joint = np.exp(
    parameters.joint_logpdf(np.hstack((xx.reshape(-1, 1), yy.reshape(-1, 1))))
).reshape(xx.shape)

fig, ax = plt.subplots()
contour = ax.contourf(xx, yy, joint, levels=20)
fig.colorbar(contour, ax=ax, label="$p(x_s, y_s)$")

ax.set_xlabel("$x_s$")
ax.set_ylabel("$y_s$")
ax.set_aspect("equal", "box")
ax.set_title("Joint distribution")
plt.show()

Nice, so we defined a joint distribution for the parameters uncertainty source position! Let us visualize some source samples! We can generate source fields through samples of $p(x_s, y_s)$:
1. Generate samples $\left(x_s^{(s)},y_s^{(s)}\right) \sim p(x_s,y_s)$
2. Compute $s(x,y,x_s^{(s)},y_s^{(s)})$

In [ ]:
# Let's define the source term


def source_term(x, y, x_s, y_s):
    return np.exp(-0.5 * ((x - x_s) ** 2 + (y - y_s) ** 2) / (0.1) ** 2)

In [ ]:
def source_field_on_mesh(source_position):
    # Returns nodal value of the source term on the domain Omega
    return source_term(mesh.p[0], mesh.p[1], source_position[0], source_position[1])

In [ ]:
# Fix the random seed
np.random.seed(42)


n_samples = 5
source_position_samples = parameters.draw_samples(n_samples)

fig, ax = plt.subplots(1, n_samples)

for i, sample in enumerate(source_position_samples):
    # Compute the source on the domain
    source_sample = source_field_on_mesh(sample)

    # Plot the source field
    plot_to_axis(source_sample, ax[i])
    ax[i].set_title(f"Source sample ${i+1}$")

fig.set_size_inches(15, 4)
fig.suptitle("Source field samples")
fig.tight_layout()

plt.show()

So we are now able to generate samples of the source field! Now let's have a look at the model outputs, i.e., the solution fields, for these samples. 


To ease notation, we define $u(x_s, y_s)$ to be the solution of the Poisson equation for a given source sample $s\left(x,y,x_s,y_s\right)$.

In [ ]:
fig, ax = plt.subplots(2, n_samples)

for i, sample in enumerate(source_position_samples):
    # Compute the source on the domain
    source_sample = source_field_on_mesh(sample)

    text_sample = "x_s^{(" + f"{i+1}" + ")}" + ", y_s^{(" + f"{i+1}" + ")}"
    # Plot the source field
    plot_to_axis(source_sample, ax[0, i])
    ax[0, i].set_title(f"Source sample $({text_sample})$")

    # Solve the Poisson equation
    solution_field = poisson_pde(sample[0], sample[1], source_term)
    plot_to_axis(solution_field, ax[1, i])
    ax[1, i].set_title("Solution $u(" + text_sample + ")$")


fig.set_size_inches(15, 6)
fig.suptitle(
    "Source field samples $s(x,y,x_s^{(s)},y_s^{(s)})$ (top row) and their respective solutions $u^{(s)}$ (bottom row)"
)
fig.tight_layout()

plt.show()

As you can see, the source position strongly dictates the resulting solution field, indicating a strong dependency between input and outputs.

Studying this dependency is a form of uncertainty quantification. Congrats!

As it becomes tedious to look at individual samples, we're looking for representative values or descriptions of the sample set. As it is common in statistics, we can look at the mean value of the solution field $u$:

$$\mu = \int u p(u) du$$

Since the distribution $p(u)$ is unknown, we employ the law of the unconscious statistician (LOTUS) to rewrite the integral as

$$\mu = \int u(x_s, y_s) p(x_s, y_s) dx_sdy_s$$

As we can see, we don't need to know $p(u)$, hence unconscious in LOTUS. However, one difficulty remains: evaluating the integral, in particular, since the solution field depends nonlinearly on the source positions. Instead, we'll have a look at numerical integration, specifically Monte Carlo integration.

## Monte Carlo integration
*Some theory.*

Monte Carlo integration approximates integrals of the form
$$ \mathbb{E}_{p(\theta)}[h] = \int h(\theta) p(\theta) d\theta$$
by 
$$ \mathbb{E}_{p(\theta)}[h] \approx \frac{1}{N} \sum_{i=1}^{N} h(\theta^{(i)})$$
where $\theta^{(i)}$ are independent and identically distributed (iid) samples of the probability distribution $p(\theta)$. Monte Carlo approaches differ from quadrature rules in two major ways:
1. A Monte Carlo estimator is a sum of random variables and, therefore, itself a random variable. Hence, the repeating Monte Carlo estimation with different samples will yield different results!
2. The expected error $\epsilon_\text{MC}$ in Monte Carlo estimation is $\epsilon_\text{MC} \propto \frac{1}{\sqrt(N)}$. Although a convergence rate of $\frac{1}{2}$ is not desirable, the expected error is independent of the dimension of the integral! This allows the employment of Monte Carlo approaches for high-dimensional integration.

*Application to our example.*

For our example, the Monte Carlo estimation of the mean value is given by
$$\mu =\mathbb{E}_{p(x_s, y_s)}[u] \approx \frac{1}{N} \sum_{i=1}^{N} u(x_s^{(i)},y_s^{(i)})$$

with $x_s^{(i)}, y_s^{(i)} \sim p(x_s, y_s)$. 

In [ ]:
# Only a wrapper as the source term is constant
def solve_poisson(x_s, y_s):
    return poisson_pde(x_s, y_s, source_term)


from queens.global_settings import GlobalSettings
from queens.drivers import Function
from queens.schedulers import Local
from queens.models import Simulation
from queens.iterators import MonteCarlo
from queens.main import run_iterator
from queens.utils.io import load_result
import pathlib

def monte_carlo_queens(n_samples, experiment_type, seed=42):
    with GlobalSettings(
        experiment_name=f"{experiment_type}_{n_samples}_seed_{seed}",
        output_dir="./output/poisson_example",
    ) as gs:
        # Driver: calls the solve_poisson function
        driver = Function(parameters, solve_poisson)

        # Scheduler: Start simulations in parallel 4 at once
        scheduler = Local(gs.experiment_name, num_jobs=4, verbose=True)

        # Model: The interface for QUEENS iterators
        model = Simulation(scheduler, driver)

        # Iterator: i.e., the analysis, in this case Monte Carlo integration
        iterator = MonteCarlo(
            model=model,
            parameters=parameters,
            global_settings=gs,
            result_description={"write_results": True, "plot_results": False},
            num_samples=n_samples,
            seed=seed,
        )

        # Run the analysis
        run_iterator(iterator, gs)

        # Load the results
        results = load_result(gs.result_file(".pickle"))

        # Return the results dict
        return results

In [ ]:
# Plot the mean value estimate with different samples

fig, ax = plt.subplots(1, 4)
n_samples = 10

for seed in range(4):
    results = monte_carlo_queens(n_samples, "monte_carlo_poisson_samples", seed)
    mean = results["mean"]

    plot_to_axis(mean, ax[seed])
    ax[seed].set_title(f"MC run {seed+1}")

fig.suptitle(
    f"Mean value for $\mu$ using Monte Carlo integration with {n_samples} samples"
)
fig.set_size_inches(15, 4)
fig.tight_layout()

plt.show()

As can be seen, with 10 samples, the mean value estimations vary quite a lot. This is the first major difference mentioned above. As mentioned, this problem can be mitigated by increasing the number of samples. Let's try:

In [ ]:
# plot the mean value for different number of samples

fig, ax = plt.subplots(4, 4)

for log10_samples in range(4):
    for seed in range(4):
        results = monte_carlo_queens(int(10**log10_samples), "monte_carlo_poisson_log_samples",seed)
        mean = results["mean"]

        plot_to_axis(mean, ax[log10_samples, seed])
        if log10_samples == 0:
            ax[log10_samples, seed].set_title(f"MC run {seed}")
        ax[log10_samples, seed].text(
            0.5,
            0.08,
            f"{int(10**log10_samples)} samples",
            ha="center",
            size="large",
            backgroundcolor="white",
        )

fig.suptitle("Mean value for $\mu$ using Monte Carlo integration")
fig.set_size_inches(16, 16)
fig.tight_layout()

plt.show()

With an increasing number of samples the variance between MC estimate of the mean value is reduced!

### Beyond mean value
Depending on the underlying distribution, the mean value might not be a quantifier of the underlying data.

> Note: As an example, think about a disease that mostly affects babies (up to 4 years) and the elderly (above 80 years); the average age of the patients observed by a hospital will be around middle-aged, which is outside of both groups.

Using the expectations, we can also obtain the probability distribution of $u_c$ at different locations $x_c, y_c$:
$$p(u_c)= \mathbb{E}_{p(x_s,y_s)}[\delta(u_c - u(x_c,y_c,x_s,y_s))]$$
where $\delta(\circ)$ is the Dirac mass delta. Numerically, this can be achieved via a kernel density estimation (KDE). The latter example can be seen here:

In [ ]:
from scipy import stats

# Indices in mesh at approximatelty x=0.2, 0.5, 0.8 and y = 0.5
index = [1969, 6, 2011]
monte_carlo_sample_outputs = results["raw_output_data"]["result"][:, index]

fig, axes = plt.subplots(1, 2)
plot_to_axis(mean, axes[0])
axes[0].set_title("Mean estimate with 1000 samples")
axes[1].set_title("PDF KDE with 1000 samples")

axes[1].set_xlabel(f"$u_c$")
axes[1].set_ylabel(f"$p(u_c)$")

min_u = monte_carlo_sample_outputs.min()
max_u = monte_carlo_sample_outputs.max()
u = np.linspace(min_u, max_u, 1000)

colors = ["r", "b", "k"]
for i, samples_at_location in enumerate(monte_carlo_sample_outputs.T):
    kde = stats.gaussian_kde(samples_at_location)
    pdf = kde.pdf(u)
    pdf[0] = 0
    pdf[-1] = 0
    pdf /= np.trapz(pdf, u)
    axes[0].plot(mesh.p[0, index[i]], mesh.p[1, index[i]], "ks")
    c_text = f"({mesh.p.T[index[i]][0]}, {mesh.p.T[index[i]][1]})"
    axes[0].text(mesh.p[0, index[i]], mesh.p[1, index[i]] + 0.02, c_text, ha="center")
    axes[1].plot(u, pdf, colors[i], label=f"$p(u_c)$ for $(x_c,y_c)={c_text}$")
    axes[1].plot([mean[index[i]]] * 2, [0, max(pdf) + 5], colors[i] + ":")
    axes[1].text(mean[index[i]], max(pdf) + 10, f"$\mu$ at ${c_text}$", color=colors[i])

axes[1].legend()

fig.suptitle("Mean value for $\mu$ using Monte Carlo integration")
fig.set_size_inches(16, 8)
fig.tight_layout()
plt.show()

Since the solution field is generated based on samples of $p(x_s, y_s)$, *random in, random out*, it can be interpreted as a random variable indexed by a spatial location $x,y$. This is known as a random field! 

Here, for each location $x_c, y_c$ we can compute a probability distribution. This is what we see on the right plot. The distribution of the solution value $u$ depends on the location $x_c, y_c$. As can be observed, the distribution shows a large variance for $x_c \approx 0.2$ and $x_c \approx 0.5$, indicating that the solution at these locations is strongly affected by the source term. In contrast, for $x_c \approx 0.8$, the probability mass, i.e., where $p(u_c)>0$, tends to concentrate around smaller values! Consequently, the variance of the solution at this location is much smaller.

# Why QUEENS?

- Even though Monte Carlo is a straightforward algorithm, utilizing QUEENS offers several advantages that enhance its functionality and usability. One significant benefit is that QUEENS provides various convenience functions for handling results and logging. For instance, if you check the folder `output/poisson_example`, you will find log files and result pickle files generated from the QUEENS runs. This organized output makes it easier to track and analyze the results of your simulations.

- Another key advantage of QUEENS is its support for parallelism. The Monte Carlo algorithm is inherently parallelizable, meaning each model evaluation can be conducted independently of the others. In the context of these examples, setting `num_jobs=4` in the scheduler allows for the execution of four simulations simultaneously. This capability significantly speeds up the computation process, making it more efficient and effective for handling large-scale simulations.

- Additionally, QUEENS offers model independence, a crucial feature for users working with various modeling frameworks. QUEENS does not require knowledge of the inner workings of the model or the specifics of libraries like scikit-fem. Instead, it seamlessly manages all the evaluation processes, allowing users to integrate their models without needing to modify QUEENS itself. 

## Let's play around

Let your creativity flow and try out stuff.

### Inspirations
- Change the parameters of the marginal distribution
- Change the distributions $p(x_s, y_s)$
- Change the source definition
- Change the number of samples
- Plot the variance of $u$ (hint, $results["var"]$)

### Some questions
- What are the downsides of Monte Carlo?
- What is the limiting factor?
- What's the variance of the outputs at the boundary?